# Starter for Cores

In this notebook we provide an explanation of the different choices for Cores. For each of them we include: 
- a written description of the Core
- a code demo how to use the Core

---

### The model has two main parts

The baseline CNN model (which is mostly based on [this work](https://openreview.net/forum?id=Tp7kI90Htd)) is constructed from two main parts:
- **core**: the core aims to (nonlinearly) extract features that are common between neurons. That is, we assume there exist a set of features that all neurons use but combine them in their own unique way.
- **readout**: once the core extracts the feautures, then a neuron reads out from those features by simply linearly combining those features into a single value. Finally, by passing this single value through a final nonlinarity (in this case `ELU() + 1`) we make sure that the model output is positive and we get the inferred firing rate of the neuron.

### Currently (March 2024) there are 4 versions of Core modules

1. Stacked2dCore
2. RotationEquivariant2dCore
3. SE2dCore
4. TransferLearningCore

Descriptions of each **Core** module can be found in the neuralpredictors/layers/cores/conv2d.py. However, for convenience we will include descriptions in each dedicated block. 

---

Before learning how to use the cores, let's create a dummy data **images**. This data will be similar to a batch of images.

<a id='another_cell'></a>
### Simulated batch of images 

In [8]:
images = torch.ones(32, 1, 144, 256)

---

Throughout the notebook we will refer to the elements of this shape in the following manner:

[1] is the number of channels (can be input, hidden, output)

[144] is the height of image or feature maps

[256] is the height of image or feature maps

[32] is the batch size, which is not as relevant for understanding the material in this notebook.

---

### Stacked2dCore

Made up of layers layers of nn.sequential modules.
The convolutional layers can be set to be either depth-separable (as used in the popular MobileNets) or based on self-attention (as used in Transformer networks). Finally, it is also possible to make this core fully linear, by disabling all nonlinearities. This makes it effectively possible to turn a core+readout CNN into a LNP-model.

Next code cell contains some configuration parameters for the stacked2d core.

In [9]:
stacked2dcore_config = {
    # core args
    'input_channels': 1,
    'input_kern': 9,
    'hidden_kern': 7,
    'hidden_channels': 64,
    'layers': 4,
    'stack': -1,
    'pad_input': True
}

(`input_channels` = 1) specifes the number of input channels. In the dataset we have loaded we have gray scale images, so the number of input channels is 1. If you have colored RGB images, you will need to set the input channels to 3. If you want your model to take into account also the behavioral parameters, such as pupil size and running speed, you accordingly increase the input channels to 5. 
(`hidden_channels` = 64) specifies the number of hidden channels. This is freely up to the user to define. If you want to have different sized hidden layers, you can pass on a list of length (`layers`) 

(`input_kern` = 9) sets the size of convolutional kernel at the first layer. 
(`hidden_kern` = 7) sets the size of convolutional kernel for all hidden layers. If you want to have different sized convolutional kernels, you can pass on a list of length (`layers`)

In [10]:
from neuralpredictors.layers.cores import Stacked2dCore 

stacked2d_core = Stacked2dCore(**stacked2dcore_config)
stacked2dcore_out = stacked2d_core(images)
print(stacked2dcore_out.shape)

torch.Size([32, 64, 144, 256])


We see that the output of the stacked2dcore has:
- **64** channels as specified by the `hidden_channels`. This implies the core ouputs only the last hidden layer, which is specified by (`stack` = -1), which is another core configuration parameter. 
- a height of **144** and a width of **256** same as input images. This implies that the images are padded with zeros to keep the same input dimensions at the output. This is specified throught the (`pad_input` = True) model configuration parameter. 

### RotationEquivariant2dCore 

A core built of 2d rotation-equivariant layers. For more info refer to https://openreview.net/forum?id=H1fU8iAqKX.

Next code cell contains some configuration parameters for RotationEquivariant2dCore. 

Because this core is built on Stacked2dCore, same configuration parameters are passed with **stacked2dcore_config**. Additionally, we set the (`num_rotations` = 8), which is the idea of RotationEquvariant CNN where feature maps at each layer are rotated. This, of course, would increase the number of output channels. 

To keep the number of output channels same, we need to adjust the number of (`hidden_channels` = 8). 

In [11]:
rotation_equivariant_2d_core_config = {
    # core args
    **stacked2dcore_config,
    'num_rotations': 8
}
rotation_equivariant_2d_core_config['hidden_channels'] = 8

In [12]:
from neuralpredictors.layers.cores import RotationEquivariant2dCore

rotationequivariant_core = RotationEquivariant2dCore(**rotation_equivariant_2d_core_config)
rotationequvariant_out = rotationequivariant_core(images)

In [13]:
rotationequvariant_out.shape

torch.Size([32, 64, 144, 256])

### SE2dCore

An extension of the Stacked2dCore class.
Additionally, a SqueezeAndExcitation layer (also called SE-block) can be added after each layer or the n final layers. For more info refer to https://arxiv.org/abs/1709.01507

In essence, Squeeze and Excitation block reweights the channels in the feature map based on their interdependecies. 

Next code cell contains some configuration parameters for SE2dCore. 

Because this core is built on Stacked2dCore, same configuration parameters are passed with **stacked2dcore_config**. Additionally, we set the (`se_reduction` = 16), which is responsible for the reduction of channels for global pooling of the Squeeze and Excitation Block. We set (`n_se_blocks` = 2), which sets the number of squeeze and excitation blocks inserted from the last layer.

Examples: layers=4, n_se_blocks=2:

=> layer0 -> layer1 -> layer2 -> SEblock -> layer3 -> SEblock

In [14]:
SE2d_core_config = {
    # core args
    **stacked2dcore_config,
    'se_reduction':16,
    'n_se_blocks':2
}

In [15]:
from neuralpredictors.layers.cores import SE2dCore

se2d_core = SE2dCore(**SE2d_core_config)
se2dcore_out = se2d_core(images)

In [16]:
se2dcore_out.shape

torch.Size([32, 64, 144, 256])

### TransferLearningCore

 Core based on popular image recognition networks from torchvision such as VGG or AlexNet. Can be instantiated as random or pretrained. Core is frozen by default, which can be changed with the fine_tune argument.

Next cell contains some configuration parameters for TransferLearning core.

TransferLearning is **not** based on Stacked2dCore, so we need new configuration parameters.

(`input_channels` = 1) 1 for gray scale images, 3 for RGB

(`tl_model_name` = 'vgg16') at the moment (March 2024) can only take models from torchvision, such as vgg16, alexnet etc.

(`layers` = -1) Number of layers, i.e. after which layer to cut the original network. More information in the next blocks.

(`pretrained` = True) Whether to use a randomly initialized or pretrained network

(`fine_tune` = False) Whether to clip gradients before this core or to allow training on the core

In [17]:
TransferLearning_vgg16_core_config = {
    # core args
    'input_channels':1,
    'tl_model_name':'vgg16',
    'layers':-1,
    'pretrained':True,
    'fine_tune':False
}

Try loading pretrained VGG16

In [18]:
from neuralpredictors.layers.cores import TransferLearningCore

transfer_learning_core = TransferLearningCore(**TransferLearning_vgg16_core_config)

To understand the `layers` parameter for the TransferLearning core, we first set (`layers` = -1) to transfer all the layers of the network.

By printing the sequential layers of the transferred network we can see the model architecture.

In [19]:
print(transfer_learning_core)

TransferLearningCore(
  (features): Sequential(
    (TransferLearning): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): 

Let's say we want to use the VGG16 model up to layer 12. Then we can change the `layers` configuration parameter in **TransferLearning_vgg16_core_config** and then re-instantiate the core.

In [20]:
TransferLearning_vgg16_core_config['layers'] = 12

In [21]:
from neuralpredictors.layers.cores import TransferLearningCore

transfer_learning_core = TransferLearningCore(**TransferLearning_vgg16_core_config)
transfer_learning_core_out = transfer_learning_core(images)
transfer_learning_core_out.shape

torch.Size([32, 256, 36, 64])

---

In conclusion, we have created instances of core models, such as **model_stacked2dcore, model_transfer_learning etc.**. After we have obtained core outpus, such as **stacked2dcore_out, transfer_learning_core_out** we can pass this as input to the readout module. 

---

### References
- Code for the [model function](https://github.com/sinzlab/sensorium/blob/8660c0c925b3944e723637db4725083f84ee28c3/sensorium/models/models.py#L17)
- Code for the [core Module](https://github.com/sinzlab/neuralpredictors/blob/0d3d793cc0e1f55ec61c5f9f7a98318b5241a2e9/neuralpredictors/layers/cores/conv2d.py#L27)